**.env 파일**

- API키 저장한 .env 파일 불러오기

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

**라이브러리 불러오기**

In [2]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain.schema import HumanMessage
from operator import itemgetter
from textwrap import dedent

In [7]:
# 경로 설정
DIRECTORY_PATH = r"Raw_DB"
PERSIST_DIRECTORY = r"Raw_DB\vector_store\contents"
COLLECTION_NAME = "contents"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

In [6]:
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

if os.path.exists(PERSIST_DIRECTORY):
    vector_store = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        collection_name=COLLECTION_NAME,
        embedding_function=embedding_model,
    )
else:
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY,
    )

print(
    "=== 내용 확인 ======================================================================================================================================"
)
try:
    results = vector_store.similarity_search("화산귀환")
    print(results[0] if results else "검색 결과 없음")
except Exception as e:
    print(f"검색 오류: {e}")

C:\Users\Playdata\AppData\Local\Temp\ipykernel_13640\722385000.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_13640\722385000.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


=== 내용 확인 ======================================================================================================================================
page_content='id: 59647809, type: 웹소설, platform: 카카오페이지, title: 화산천마, status: 월~금 연재, thumbnail: https://page-images.kakaoentcdn.com/download/resource?kid=vPpRR/hAHIq1vh21/tedW3p9mEjLVCGyeMXNXBk&filename=o1/dims/resize/384, genre: 무협, views: 52453000, rating: 9.2, like: -, description: 천마신교의 교주이자, 천하제일인이라 불리는 절대천마. 영생불멸군림대법으로 인해 정파에서 깨어나고 마는데……. "뭐? 화산파? 이건 또 무슨 개소리야?" 갑자기 화산파의 제자가 된 마도 대종사의 이야기. 몰락의 길을 걷던 화산파가 다시 비상하기 시작한다., keyword: ['퓨전', '환생', '먼치킨', '고인물', '천마', '도사', '정파', '검', '화산'], author: 사야객, illustrator: -, original: -, age_rating: 전체이용가, price_type: 1일 기다리면 무료, price_price: -' metadata={'age_rating': '전체이용가', 'author': '사야객', 'description': '천마신교의 교주이자, 천하제일인이라 불리는 절대천마. 영생불멸군림대법으로 인해 정파에서 깨어나고 마는데……. "뭐? 화산파? 이건 또 무슨 개소리야?" 갑자기 화산파의 제자가 된 마도 대종사의 이야기. 몰락의 길을 걷던 화산파가 다시 비상하기 시작한다.', 'episode': 777, 'genre': '무협', 'id': 59647809, '

In [8]:
# Retriever 설정 - 검색 설정
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,
        "fetch_k": 10,
        "lambda_mult": 0.2,
    },
)

print(f"vector store가 {PERSIST_DIRECTORY}에서 성공적으로 생성/로드되었습니다.")

vector store가 Raw_DB\vector_store\contents에서 성공적으로 생성/로드되었습니다.


In [19]:
# db 검색 tool
@tool
def search_contents(query: str) -> list[Document]:
    """
    Vector Store에 저장된 웹툰,웹소설 정보를 검색한다.
    이 도구는 웹툰 추천, 웹소설 추천에 대해 실행한다.
    """
    result = retriever.invoke(query)
    return result if result else [Document(page_content="검색 결과가 없습니다.")]


# web 검색 tool
@tool
def search_web(query: str) -> list[Document]:
    """
    Web에서 웹툰 추천,웹소설 추천 정보를 검색한다.
    이 도구는 웹툰 추천, 웹소설 추천에 대해 실행한다.
    """
    try:
        tavily_search = TavilySearchResults(max_results=2)
        result = tavily_search.invoke(query)
        if result:
            return [
                Document(
                    page_content=item.get("content", ""),
                    metadata={"title": item.get("title", "")},
                )
                for item in result
            ]
        else:
            return [Document(page_content="검색 결과가 없습니다.")]
    except Exception as e:
        return [Document(page_content=f"오류 발생: {str(e)}")]

In [43]:
# LLM 구성
prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("agent_scratchpad"),
        (
            "ai",
            dedent(
                """
        1. 너는 웹툰,웹소설을 맨날 보는 매니아야.
        2. 친구가 추천해달라고 하면 친구 취향에 꼭 맞는 웹툰을 추천해주면 되는거야. 
        3. 너가 잘못 추천해주면 친구가 웹툰에 대한 흥미를 잃을 수 있어. 신중하게 선택해.
        답변 작성에 대한 지침은 다음과 같습니다.

        1. 없는 정보를 창조해서 답변하지 마. 
        2. 네이버, 네이버 시리즈, 카카오웹툰, 카카오페이지에 있는 웹툰, 웹소설 만 추천해줘.
        3. 입력 메세지로 사용자의 취향을 모르겠다면 유도 질문을 해. 예시) 재밌는 웹툰 추천 해줘. -> 주로 어떤 장르를 재밌게 보시나요?
        4. 최소 8개, 최대 15개를 추천해주세요.
        5. 사용자의 성향을 이해할 때는 장르보다 구체적이어야 합니다. 예) 키워드 조합


        위 안내에 따라 사용자의 성향에 맞는 웹툰이나 웹소설을 추천해주세요.
        {context}
    """
            ),
        ),
        MessagesPlaceholder("history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory()  # 메모리 설정
model = ChatOpenAI(model="gpt-4o", temperature=0)
parser = StrOutputParser()

# agent 구성
agent = create_tool_calling_agent(
    llm=model, tools=[search_contents, search_web], prompt=prompt_template
)

runnable = (
    {
        "context": RunnableLambda(lambda x: retriever.invoke(x["question"])),
        "question": itemgetter("question"),
        "history": itemgetter("history"),
    }
    | prompt_template
    | model
    | parser
)

chain = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=lambda session_id: memory.chat_memory,
    input_messages_key="question",
    history_messages_key="history",
)


In [44]:
toolkit = [search_contents, search_web]

agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

In [45]:
# 사용자 질문
query = input("질문을 입력하세요: ").strip()
if not query:
    print("유효한 질문을 입력하세요.")
    exit()

In [46]:
# context와 history 기본값 설정
context = "기본 context가 비어 있습니다. 적절한 데이터를 제공하세요."
history = []


result_from_db = search_contents.invoke(query)
result_from_web = search_web.invoke(query)

In [47]:
def combine_search_results(result_from_db: list, result_from_web: list) -> str:
    combined_context = [
        "저장된 데이터에서 찾은 정보:\n",
        *[doc.page_content for doc in result_from_db],
        "실시간 web 검색에서 확인된 정보:\n",
    ]
    if result_from_web:
        combined_context.extend(
            [f"[{idx}] {doc.metadata.get('title', '제목 없음')}: {doc.page_content}"
            for idx, doc in enumerate(result_from_web, start=1)]
        )
    else:
        combined_context.append("web 검색 결과가 없습니다.")
    return "\n".join(combined_context)

combined_context = combine_search_results(result_from_db, result_from_web)

In [48]:
# LLM 입력 메세지 구성
final_input_messages = [
    HumanMessage(content=f"사용자 질문: {query}"),
    HumanMessage(content=f"context: {combined_context}")
]

In [49]:
# 최종 응답 생성
final_response = agent_executor.invoke(
    {
        "question": query,
        "context": combined_context,
        "history": history,
    }
)

print(final_response["output"])



> Entering new AgentExecutor chain...

Invoking: `search_contents` with `{'query': '짝사랑 로맨스 판타지 웹툰'}`


[Document(metadata={'age_rating': '전체연령가', 'author': '루시드', 'description': '사랑을 알고 싶은 호러 작가와 \n공포물은 질색인 인기 로맨스 작가의 \n오싹달콤한 로맨스 제작기', 'episode': 75, 'genre': '로맨스', 'id': 710748, 'illustrator': '루시드', 'keywords': '로맨스, 완결로맨스', 'like': 118425, 'original': '-', 'platform': '네이버 웹툰', 'price_price': '-', 'price_type': '24시간 기다리면 무료', 'rating': 9.97503, 'status': '75화 완결', 'thumbnail': 'https://image-comic.pstatic.net/webtoon/710748/thumbnail/thumbnail_IMAG21_7017792614173860918.jpg', 'title': '호러와 로맨스', 'type': '웹툰', 'url': 'https://comic.naver.com/webtoon/list?titleId=710748', 'views': '-'}, page_content="id: 710748, type: 웹툰, platform: 네이버 웹툰, title: 호러와 로맨스, status: 75화 완결, thumbnail: https://image-comic.pstatic.net/webtoon/710748/thumbnail/thumbnail_IMAG21_7017792614173860918.jpg, genre: 로맨스, views: -, rating: 9.97503, like: 118425, description: 사랑을 알고 싶은 호러 작가와 \n공포물은 질색인 인기 로맨스 작가